In [ ]:
import os
import numpy as np
import time
import sys
import csv
import cv2
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.metrics.ranking import roc_auc_score
import sklearn.metrics as metrics
import random

In [ ]:
image_set_csv = 'CheXpert-v1.0-small/train.csv'
image_set = 'CheXpert-v1.0-small/train'

test_set_csv = 'CheXpert-v1.0-small/valid.csv'
test_set = 'CheXpert-v1.0-small/valid'

use_gpu = torch.cuda.is_available()

pretrained = True                
classes = 14     
batch_size = 16
epochs = 20
resize_im = 224
#'Enlarged Cardiomediastinum', 'Pneumonia', 'Atelectasis', 

# class_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 
#                'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 
#                'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

class_names = ['Enlarged Cardiomediastinum']
# pathology = 'Atelectasis'


In [ ]:
df = pd.read_csv(image_set_csv)
# df=df.fillna(0)
# df_ran = df[['Path','Enlarged Cardiomediastinum']]
# print(df[['Path','Enlarged Cardiomediastinum']].head())
# print(len(df[df['Enlarged Cardiomediastinum']==-1]))
# print(df_ran.iloc[1,0])
# # df=df.fillna(0)
# # label = int(df.loc[df['Path']=='CheXpert-v1.0-small/train/patient29525/study7/view1_frontal.jpg','Enlarged Cardiomediastinum'])
# # print(label)
# # print(type(label))

In [ ]:
X = pd.read_csv(image_set_csv)
X = X.fillna(0)
# print(X.shape)
# print(X.loc[:,X.columns!=pathology].head())
X_train, X_test, y_train, y_test = train_test_split(X.loc[:,X.columns!=pathology], X[pathology], test_size=0.20, random_state=42,stratify = X[pathology])
print(len(X_train))
print(len(X_test))
X = X_train.join(y_train)
print(X.shape)

In [ ]:
class Siamese_CheXpertDataSet(Dataset):
    def __init__(self, image_set_csv, image_set,set_type, pathology, transform=None):
        """
        image_list_file: path to the file containing images with corresponding labels.
        transform: optional transform to be applied on a sample.
        """
        df_train = pd.read_csv(image_set_csv)
        df_train = df_train.fillna(0)
        if set_type == 'train' or set_type == 'val':
            X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:,df_train.columns!=pathology], df_train[pathology], test_size=0.20, random_state=42, stratify = df_train[pathology])
            X_train = X_train.join(y_train)
            X_val = X_val.join(y_val)

        if set_type == 'train':
            self.df = X_train[['Path',pathology]]
            self.images_names = list(X_train["Path"])
            
        elif set_type == 'val':
            self.df = X_val[['Path',pathology]]
            self.images_names = list(X_val["Path"])
        else: 
            self.df = df_train[['Path',pathology]]
            self.labels = list(df_train[pathology])
            self.images_names = list(df_train["Path"])

        self.transform = transform
        self.set_type = set_type

    def __getitem__(self, index):
        """Take the index of item and returns the image and its labels"""
        #Assuming -1 are equivalent to 0
        df_ones = self.df[self.df[pathology]==1]
        df_ones = pd.concat([df_ones, self.df[self.df[pathology]==-1]])
        df_others = self.df[self.df[pathology]==0]
        index = np.random.randint(0,len(self.images_names))
        same_class = np.random.randint(0,2) 
        anchor_class = np.random.randint(0,2) 
        #image_names = df_train[:,[pathology]] == 1
        # .5 for uncertainty?
        if self.set_type == 'val' or self.set_type == 'train':
            if same_class:
                if anchor_class:
                    image = np.random.choice(df_ones['Path'],2)
                    image_1 = image[0]
                    image_2 = image[1]
                else:
                    image = np.random.choice(df_others['Path'],2)
                    image_1 = image[0]
                    image_2 = image[1]
                label = 1
            else:
                image_1 = np.random.choice(df_ones['Path'])
                image_2 = np.random.choice(df_others['Path'])
                label = 0

            image_1 = Image.open(image_1).convert('RGB')
            image_2 = Image.open(image_2).convert('RGB')
            if self.transform is not None:
                image_1 = self.transform(image_1)
                image_2 = self.transform(image_2)
            return image_1, image_2, label
    
        else:
            image_name = self.images_names[index]
            image = Image.open(image_name).convert('RGB')
            label = self.labels[index]
            if self.transform is not None:
                image = self.transform(image)
            return image, label


    def __len__(self):
        return len(self.images_names)

In [ ]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList= []
transformList.append(transforms.RandomVerticalFlip())
transformList.append(transforms.Resize((resize_im, resize_im)))
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence_train=transforms.Compose(transformList)

transformList= []
transformList.append(transforms.Resize((resize_im, resize_im)))
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence_val=transforms.Compose(transformList)

In [ ]:
pathology = 'Enlarged Cardiomediastinum'
datasetTrain = Siamese_CheXpertDataSet(image_set_csv,image_set,'train',pathology,transformSequence_train)
datasetVal = Siamese_CheXpertDataSet(image_set_csv,image_set,'val',pathology,transformSequence_val)
datasetTest = Siamese_CheXpertDataSet(test_set_csv,test_set,'test',pathology,transformSequence_val)

dataLoaderTrain = DataLoader(dataset=datasetTrain, batch_size=batch_size, shuffle=True,  num_workers=24, pin_memory=True)
dataLoaderVal = DataLoader(dataset=datasetVal, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
dataLoaderTest = DataLoader(dataset=datasetTest, batch_size=1, shuffle=False, num_workers=24, pin_memory=True)

In [ ]:
print(len(dataLoaderTrain))
print(len(dataLoaderVal))
print(len(dataLoaderTest))
print(len(datasetTrain))
print(len(datasetVal))
print(len(datasetTest))
# print(np.random.randint(0,2))

In [ ]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()  
dataiter = iter(dataLoaderTrain)

example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy())

In [ ]:
class ContrastiveLoss(torch.nn.Module):

      def __init__(self, margin):
            super(ContrastiveLoss, self).__init__()
            self.margin = margin
            self.eps = 1e-9

      def forward(self, output1, output2, label):
#             print(output1.shape)
#             print(output2.shape)
#             print(label.shape)
#             print(label)
            # Find the pairwise distance or eucledian distance of two output feature vectors
            euclidean_distance = F.pairwise_distance(output1, output2)
            # perform contrastive loss calculation with the distance
            loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
            (label) * torch.pow(torch.clamp(self.margin - (euclidean_distance+self.eps), min=0.0), 2))

            return loss_contrastive
        
loss = ContrastiveLoss(margin=10)

In [ ]:
class Siamese_DenseNet121(nn.Module):
    def __init__(self, classNum, pretrained):

        super(Siamese_DenseNet121, self).__init__()

        self.densenet121 = torchvision.models.densenet121(pretrained=pretrained, memory_efficient=True)
        prevNum = self.densenet121.classifier.in_features
        #replacing classifier FC for 1 class
        self.densenet121.classifier = nn.Sequential(nn.Linear(prevNum, classNum), nn.ReLU())

    def forward_once(self, x):
        x = self.densenet121(x)
        return x

    def forward(self, input1, input2):
        output1 = self.densenet121(input1)
        output2 = self.densenet121(input2)
        return output1, output2

In [ ]:
class Siamese_ChexpertTrainer():
 
    def train (dataloaderTrain, dataloaderVal, pretrained, classes, batch_size, epoch, resize_im, launchTimestamp, checkpoint):

        
        #-------------------- SETTINGS: NETWORK ARCHITECTURE
        #model = DenseNet121(classes, pretrained).cuda()
        model = Siamese_DenseNet121(classes, pretrained).cuda()
        # model = torch.nn.DataParallel(model).cuda()

        #-------------------- SETTINGS: OPTIMIZER & SCHEDULER
        optimizer = optim.Adam (model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
        scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 5, mode = 'min')
        
        #---- Load checkpoint 
        if checkpoint != None and use_gpu:
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])
            optimizer.load_state_dict(modelCheckpoint['optimizer'])

        
        #---- TRAIN THE NETWORK
        
        lossMIN = 100000
        
        for epochID in range (0, epoch):
            
            timestampTime = time.strftime("%H%M%S")
            timestampDate = time.strftime("%d%m%Y")
            timestampSTART = timestampDate + '-' + timestampTime
                         
            Siamese_ChexpertTrainer.epochTrain(model, dataLoaderTrain, optimizer, scheduler, epoch, classes, loss)
            lossVal, losstensor = Siamese_ChexpertTrainer.epochVal (model, dataLoaderVal, optimizer, scheduler, epoch, classes, loss)
            
            timestampTime = time.strftime("%H%M%S")
            timestampDate = time.strftime("%d%m%Y")
            timestampEND = timestampDate + '-' + timestampTime
            
            scheduler.step(losstensor)
            
            if lossVal < lossMIN:
                lossMIN = lossVal    
                torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, 'siamese_models/'+'m-' + launchTimestamp + '.pth.tar')
                print ('Epoch [' + str(epochID + 1) + '] [save] [' + timestampEND + '] loss= ' + str(lossVal))
            else:
                print ('Epoch [' + str(epochID + 1) + '] [----] [' + timestampEND + '] loss= ' + str(lossVal))
                     
    #-------------------------------------------------------------------------------- 
       
    def epochTrain (model, dataLoader, optimizer, scheduler, epochMax, classCount, loss):
        
        model.train()
        
        for batchID, (input_1, input_2, target) in enumerate (dataLoader):
                        
            input_1 = Variable(input_1).cuda()
            input_2 = Variable(input_2).cuda()
            target = Variable(target).cuda()
            
            optimizer.zero_grad()
                         
            varOutput_1,varOutput_2 = model.forward(input_1,input_1)
            lossvalue = loss(varOutput_1,varOutput_2,target)       
            lossvalue.backward()
            optimizer.step()
            
    #-------------------------------------------------------------------------------- 
        
    def epochVal (model, dataLoader, optimizer, scheduler, epochMax, classCount, loss):
        
        model.eval()
        
        lossVal = 0
        lossValNorm = 0
        
        losstensorMean = 0
        
        with torch.no_grad(): 
            
            for i, (input_1,input_2, target) in enumerate (dataLoader):

                input_1 = Variable(input_1).cuda()
                input_2 = Variable(input_2).cuda()
                target = Variable(target).cuda()

                varOutput_1,varOutput_2 = model.forward(input_1,input_1)

                losstensor = loss(varOutput_1,varOutput_2,target)
                losstensorMean += float(losstensor)

                lossVal += float(losstensor) #or losstensor.item()
                lossValNorm += 1

            outLoss = lossVal / lossValNorm
            losstensorMean = losstensorMean / lossValNorm

            return outLoss, losstensorMean
    
    def computeAUROC (dataGT, dataPRED):
        
        return roc_auc_score(dataGT, dataPRED)

            
    def test (dataloaderTest, pathModel, classes, pretrained, batch_size, resize_im, launchTimeStamp, class_names, train_set, transform):   
        
        cudnn.benchmark = True

        df_ones = train_set[train_set[pathology]==1].head(3)
        df_minus_ones = train_set[train_set[pathology]==-1].head(3) 
        df_others = train_set[train_set[pathology]==0].head(3)
        
        #-------------------- SETTINGS: NETWORK ARCHITECTURE, MODEL LOAD
        model = Siamese_DenseNet121(classes, pretrained).cuda()
        
        modelCheckpoint = torch.load(pathModel)
        model.load_state_dict(modelCheckpoint['state_dict'])
        
        outGT = []
        outPRED = []
        outPRED_ones = 0
        outPRED_zeros = 0
       
        model.eval()        

        for i, (input_2, target) in enumerate(dataLoaderTest):
            
            outGT.extend(target.numpy())
            
            if i%50 == 0:
                print("i")
                print(i)
            
            
            for j in range(3):

                input_1 = transform(Image.open(df_ones.iloc[j,0]).convert('RGB'))
                input_3 = transform(Image.open(df_minus_ones.iloc[j,0]).convert('RGB'))
                input_4 = transform(Image.open(df_others.iloc[j,0]).convert('RGB'))

                input_1 = Variable(input_1).cuda()
                input_2 = Variable(input_2).cuda()
                input_3 = Variable(input_3).cuda()
                input_4 = Variable(input_4).cuda()
                bs, c, h, w = input_2.size()
                input_1 = input_1.view(-1, c, h, w)
                input_2 = input_2.view(-1, c, h, w)
                input_3 = input_3.view(-1, c, h, w)
                input_4 = input_3.view(-1, c, h, w)
                
#                 print(type(target.numpy()))
#                 print(type(Variable(target).float()))
#                 print(type(outGT))

                out_1,out_2 = model(input_1,input_2)
                out = F.pairwise_distance(out_1, out_2)
#                 print(type(out))
#                 print(type(out.item()))
#                 print(out.shape)
                outPRED_ones += out.item()

                out_1,out_2 = model(input_3,input_2)
                out = F.pairwise_distance(out_1, out_2)
                outPRED_ones += out.item()

                out_1,out_2 = model(input_4,input_2)
                out = F.pairwise_distance(out_1, out_2)
#                 print(out.shape)
                outPRED_zeros += out.item()
            
            outPRED_ones = np.exp(outPRED_ones) / (np.exp(outPRED_ones)+np.exp(outPRED_zeros))
            outPRED_zeros = np.exp(outPRED_zeros) / (np.exp(outPRED_ones)+np.exp(outPRED_zeros))
            
#             print("outpred_ones")
#             print(outPRED_ones)
            
            if outPRED_ones > outPRED_zeros:
                outPRED.append(1)
                #outPRED = torch.cat((outPRED, 1), 0)
            else:
                outPRED.append(0)
                #outPRED = torch.cat((outPRED, 0), 0)
        
        print(len(outPRED))
        print(len(outGT))
        print(outPRED[0])
        print(outGT[0])

        aurocIndividual = Siamese_ChexpertTrainer.computeAUROC(outGT, outPRED)
        aurocMean = np.array(aurocIndividual).mean()
        
        print ('AUROC mean ', aurocMean)
        
#         for i in range (0, len(aurocIndividual)):
#             print (class_names[i], ' ', aurocIndividual[i])
        
        return outGT, outPRED, aurocMean
#-------------------------------------------------------------------------------- 

In [ ]:
timestampTime = time.strftime("%H%M%S")
timestampDate = time.strftime("%d%m%Y")
timestampLaunch = timestampDate + '-' + timestampTime
       
pathModel = 'm-' + timestampLaunch + '.pth.tar'
    
print ('Training NN architecture')
Siamese_ChexpertTrainer.train(dataLoaderTrain, dataLoaderVal, pretrained, classes, batch_size, epochs, resize_im, timestampLaunch, None)


In [ ]:
print(os.listdir('siamese_models/'))

In [ ]:
pathModel = 'siamese_models/m-30112020-044136.pth.tar'
print ('Testing the trained model')
outGT1, outPRED1, auc = Siamese_ChexpertTrainer.test(dataLoaderTest, pathModel, classes, pretrained, batch_size, resize_im, timestampLaunch, class_names,df, transformSequence_val)

In [ ]:
false_rate, true_rate, threshold = metrics.roc_curve(outGT1, outPRED1)
roc_auc = metrics.auc(false_rate, true_rate)
i=0    
plt.title('ROC for: ' + class_names[i])
plt.plot(false_rate, true_rate, label = 'U-ones: AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.savefig('Enlarged.png')

In [ ]:
plt.hist(outGT1)
plt.show()
plt.hist(outPRED1)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

labels = ['negative','positive']
cm = confusion_matrix(outGT1, outPRED1)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm,cmap=plt.cm.Blues)
plt.title('Confusion matrix for Enlarged Cardiomediastinum')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_Enlarged Cardiomediastinum.png')
plt.show()
